# Парсер статей ГК РФ

In [80]:
from bs4 import BeautifulSoup
import pathlib, re, pandas as pd

In [81]:
def read_file_text(path: pathlib.Path) -> str:
    """Reads a .doc file saved as HTML and returns plain text"""
    data = path.read_bytes()

    if b'<html' in data.lower():
        soup = BeautifulSoup(data, 'html.parser')
        text = soup.get_text('\n')
    else:
        try:
            text = data.decode('utf-8')
        except UnicodeDecodeError:
            text = data.decode('cp1251', errors='ignore')
    return text.replace('\u00A0', ' ')

In [82]:
# Regular Expression Patterns
re_section = re.compile(r'^РАЗДЕЛ\s+([IVXLCDM]+)\.?\s*(.*)$', re.I | re.M)
re_chapter = re.compile(r'^Глава\s+(\d+)\.?\s*(.*)$', re.I | re.M)
re_article = re.compile(r'^Статья\s+(\d+)\.?\s*(.*)$', re.I | re.M)

In [83]:
# Roman Numeral Converter
def roman_to_int(s: str) -> int:
    vals = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)
    total = 0
    prev = 0
    for ch in reversed(s.upper()):
        v = vals.get(ch, 0)
        total += -v if v < prev else v
        prev = v
        
    return total or None

In [84]:
# Main function for parsing
def parse_articles(path: pathlib.Path):
    text = read_file_text(path)
    lines = text.splitlines(keepends=True)

    articles_meta = []
    current_section_title = ''
    current_chapter_title = ''
    article_num = None
    article_title = ''
    article_start_char = None
    offset = 0

    for raw_line in lines:
        line = raw_line.strip()

        m_sec = re_section.match(line)
        if m_sec:
            if article_num is not None:
                article_end_char = offset
                article_text = text[article_start_char:article_end_char]
                articles_meta.append({
                    'file': path.name,
                    'article_num': article_num,
                    'article_title': article_title,
                    'article_text': article_text,
                    'section_title': current_section_title,
                    'chapter_title': current_chapter_title,
                    'start_char': article_start_char,
                    'end_char': article_end_char,
                    'text_length': len(article_text)
                })
                article_num = None
            current_section_title = m_sec.group(2).strip()

        m_ch = re_chapter.match(line)
        if m_ch:
            if article_num is not None:
                article_end_char = offset
                article_text = text[article_start_char:article_end_char]
                articles_meta.append({
                    'file': path.name,
                    'article_num': article_num,
                    'article_title': article_title,
                    'article_text': article_text,
                    'section_title': current_section_title,
                    'chapter_title': current_chapter_title,
                    'start_char': article_start_char,
                    'end_char': article_end_char,
                    'text_length': len(article_text)
                })
                article_num = None
            current_chapter_title = m_ch.group(2).strip()

        m_art = re_article.match(line)
        if m_art:
            if article_num is not None:
                article_end_char = offset
                article_text = text[article_start_char:article_end_char]
                articles_meta.append({
                    'file': path.name,
                    'article_num': article_num,
                    'article_title': article_title,
                    'article_text': article_text,
                    'section_title': current_section_title,
                    'chapter_title': current_chapter_title,
                    'start_char': article_start_char,
                    'end_char': article_end_char,
                    'text_length': len(article_text)
                })
            article_num = int(m_art.group(1))
            article_title = m_art.group(2).strip()
            article_start_char = offset

        offset += len(raw_line)

    if article_num is not None:
        article_end_char = len(text)
        article_text = text[article_start_char:article_end_char]
        articles_meta.append({
            'file': path.name,
            'article_num': article_num,
            'article_title': article_title,
            'article_text': article_text,
            'section_title': current_section_title,
            'chapter_title': current_chapter_title,
            'start_char': article_start_char,
            'end_char': article_end_char,
            'text_length': len(article_text)
        })

    return articles_meta

In [85]:
data_dir = pathlib.Path('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset')
files = sorted(data_dir.glob('Part*.doc'))  # Search Part1.doc, Part2.doc, Part3.doc, Part4.doc

In [86]:
files

[PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part1.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part2.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part3.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part4.doc')]

In [87]:
all_articles = []
for f in files:
    all_articles.extend(parse_articles(f))

df = pd.DataFrame(all_articles)
print(df.head())

        file  article_num                                      article_title  \
0  Part1.doc            1      Основные начала гражданского законодательства   
1  Part1.doc            2  Отношения, регулируемые гражданским законодате...   
2  Part1.doc            3  Гражданское законодательство и иные акты, соде...   
3  Part1.doc            4  Действие гражданского законодательства во времени   
4  Part1.doc            5                                             Обычаи   

                                        article_text    section_title  \
0  Статья 1. Основные начала гражданского законод...  Общие положения   
1  Статья 2. Отношения, регулируемые гражданским ...  Общие положения   
2  Статья 3. Гражданское законодательство и иные ...  Общие положения   
3  Статья 4. Действие гражданского законодательст...  Общие положения   
4  Статья 5. Обычаи \n(Наименование в редакции Фе...  Общие положения   

                  chapter_title  start_char  end_char  text_length  
0  Гражданс

## Check if data was parsed correctly

In [88]:
print("Shape: ", df.shape)
print("Columns: ", df.columns)

Shape:  (1735, 9)
Columns:  Index(['file', 'article_num', 'article_title', 'article_text', 'section_title',
       'chapter_title', 'start_char', 'end_char', 'text_length'],
      dtype='object')


In [89]:
for col in df.columns:
    print(f"Уникальные значения в столбце '{col}':")
    print(df[col].unique())
    print("-" * 40)


Уникальные значения в столбце 'file':
['Part1.doc' 'Part2.doc' 'Part3.doc' 'Part4.doc']
----------------------------------------
Уникальные значения в столбце 'article_num':
[   1    2    3 ... 1539 1540 1541]
----------------------------------------
Уникальные значения в столбце 'article_title':
['Основные начала гражданского законодательства'
 'Отношения, регулируемые гражданским законодательством'
 'Гражданское законодательство и иные акты, содержащие нормы гражданского права'
 ... 'Исключительное право на коммерческое обозначение'
 'Действие исключительного права на коммерческое обозначение'
 'Соотношение права на коммерческое обозначение с правами на фирменное наименование и товарный знак']
----------------------------------------
Уникальные значения в столбце 'article_text':
['Статья 1. Основные начала гражданского законодательства\n \n1. Гражданское законодательство основывается на признании равенства участников регулируемых им отношений, неприкосновенности собственности, свобод

In [90]:
# Checking for empty or NaN lines
print(df.isnull().sum())

for col in df.columns:
    num_empty = (df[col].astype(str).str.strip() == '').sum()
    print(f"Пустых строк в '{col}': {num_empty}")


file             0
article_num      0
article_title    0
article_text     0
section_title    0
chapter_title    0
start_char       0
end_char         0
text_length      0
dtype: int64
Пустых строк в 'file': 0
Пустых строк в 'article_num': 0
Пустых строк в 'article_title': 225
Пустых строк в 'article_text': 0
Пустых строк в 'section_title': 333
Пустых строк в 'chapter_title': 24
Пустых строк в 'start_char': 0
Пустых строк в 'end_char': 0
Пустых строк в 'text_length': 0


In [93]:
empty_titles = df[df['article_title'].astype(str).str.strip() == '']
print(empty_titles[['article_title', 'article_text', 'chapter_title']].iloc[:20])

    article_title                                       article_text  \
8                  Статья 8\n1\n. Государственная регистрация пра...   
17                 Статья 16\n1\n. Компенсация ущерба, причиненно...   
52                 Статья 50\n1\n. Решение об учреждении юридичес...   
56                 Статья 53\n1\n. Ответственность лица, уполномо...   
57                 Статья 53\n2\n. Аффилированность\n \nВ случаях...   
65                 Статья 60\n1\n. Последствия признания недейств...   
66                 Статья 60\n2\n. Признание реорганизации корпор...   
71                 Статья 64\n1\n. Защита прав кредиторов ликвиди...   
72                 Статья 64\n2\n. Прекращение недействующего юри...   
74                 Статья 65\n1\n. Корпоративные и унитарные юрид...   
75                 Статья 65\n2\n. Права и обязанности участников...   
76                 Статья 65\n3\n. Управление в корпорации\n \n1....   
78                 Статья 66\n1\n. Вклады в имущество хозяйствен